In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sys
import time
from functools import wraps

import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

from tensorflow.keras.layers import (
    Activation,
    Conv1D,
    Conv2D,
    Dense,
    GlobalAveragePooling2D,
    MaxPooling2D,
    Flatten,
)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import set_random_seed


<IPython.core.display.Javascript object>

In [3]:
def timing(f):
    @wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        val = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Function {f.__qualname__} finished in {end-start:.2f} seconds.")
        return val

    return inner

<IPython.core.display.Javascript object>

# Data

In [4]:
data = load_digits()

X = data["data"] / 256
y = data["target"]

<IPython.core.display.Javascript object>

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.1, random_state=42
)

<IPython.core.display.Javascript object>

In [6]:
NEW_SHAPE_TRAIN = (X_train.shape[0], 8, 8, 1)
NEW_SHAPE_TEST = (X_test.shape[0], 8, 8, 1)

X_train_reshaped = X_train.reshape(NEW_SHAPE_TRAIN)
X_test_reshaped = X_test.reshape(NEW_SHAPE_TEST)

<IPython.core.display.Javascript object>

# Create model

In [7]:
set_random_seed(42)
model = Sequential(
    [
        Conv2D(
            filters=4,
            kernel_size=(1, 1),
            padding="valid",
            strides=1,
            input_shape=((8, 8, 1)),
        ),
        Flatten(),
        #         Dense(units=15, activation="relu"),
        Dense(units=10, activation="softmax"),
    ]
)

LOSS = SparseCategoricalCrossentropy()
METRICS = ["accuracy"]
OPTIMIZER = Adam(learning_rate=0.001)


model.compile(loss=LOSS, metrics=METRICS, optimizer=OPTIMIZER)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 4)           8         
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 10)                2570      
                                                                 
Total params: 2,578
Trainable params: 2,578
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [8]:
@timing
def train(x, y, **kwargs):
    history = model.fit(x, y, **kwargs)
    return history

<IPython.core.display.Javascript object>

In [9]:
early_stopping = EarlyStopping(
    mode="auto",
    monitor="val_loss",
    patience=3,
    min_delta=0,
    restore_best_weights=False,
    verbose=1,
)

reduce_lr = ReduceLROnPlateau(
    mode="auto",
    monitor="val_loss",
    factor=0.1,
    patience=1,
    min_delta=0,
    min_lr=1e-08,
    cooldown=0,
    verbose=1,
)

tensorboard = TensorBoard(log_dir="./logs_tensorboard", write_graph=True)

<IPython.core.display.Javascript object>

In [10]:
history = train(
    X_train_reshaped,
    y_train,
    validation_data=(X_test_reshaped, y_test),
    batch_size=64,
    epochs=500,
    callbacks=[reduce_lr, early_stopping, tensorboard],
)

Epoch 1/500
26/26 [==============================] - 3s 13ms/step - loss: 2.2954 - accuracy: 0.1515 - val_loss: 2.2807 - val_accuracy: 0.3278 - lr: 0.0010
Epoch 2/500
26/26 [==============================] - 0s 4ms/step - loss: 2.2716 - accuracy: 0.3970 - val_loss: 2.2557 - val_accuracy: 0.5333 - lr: 0.0010
Epoch 3/500
26/26 [==============================] - 0s 5ms/step - loss: 2.2450 - accuracy: 0.5665 - val_loss: 2.2272 - val_accuracy: 0.6611 - lr: 0.0010
Epoch 4/500
26/26 [==============================] - 0s 4ms/step - loss: 2.2149 - accuracy: 0.6729 - val_loss: 2.1948 - val_accuracy: 0.7222 - lr: 0.0010
Epoch 5/500
26/26 [==============================] - 0s 5ms/step - loss: 2.1813 - accuracy: 0.7570 - val_loss: 2.1586 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 6/500
26/26 [==============================] - 0s 5ms/step - loss: 2.1438 - accuracy: 0.8312 - val_loss: 2.1193 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/500
26/26 [==============================] - 0s 5ms/step - loss: 2.

Epoch 54/500
26/26 [==============================] - 0s 5ms/step - loss: 0.4283 - accuracy: 0.9314 - val_loss: 0.4185 - val_accuracy: 0.9278 - lr: 0.0010
Epoch 55/500
26/26 [==============================] - 0s 5ms/step - loss: 0.4185 - accuracy: 0.9301 - val_loss: 0.4087 - val_accuracy: 0.9278 - lr: 0.0010
Epoch 56/500
26/26 [==============================] - 0s 5ms/step - loss: 0.4093 - accuracy: 0.9326 - val_loss: 0.4007 - val_accuracy: 0.9278 - lr: 0.0010
Epoch 57/500
26/26 [==============================] - 0s 5ms/step - loss: 0.4005 - accuracy: 0.9326 - val_loss: 0.3918 - val_accuracy: 0.9333 - lr: 0.0010
Epoch 58/500
26/26 [==============================] - 0s 5ms/step - loss: 0.3920 - accuracy: 0.9332 - val_loss: 0.3846 - val_accuracy: 0.9389 - lr: 0.0010
Epoch 59/500
26/26 [==============================] - 0s 5ms/step - loss: 0.3838 - accuracy: 0.9320 - val_loss: 0.3756 - val_accuracy: 0.9389 - lr: 0.0010
Epoch 60/500
26/26 [==============================] - 0s 5ms/step - lo

Epoch 107/500
26/26 [==============================] - 0s 5ms/step - loss: 0.1904 - accuracy: 0.9561 - val_loss: 0.1991 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 108/500
26/26 [==============================] - 0s 6ms/step - loss: 0.1885 - accuracy: 0.9579 - val_loss: 0.1982 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 109/500
26/26 [==============================] - 0s 5ms/step - loss: 0.1866 - accuracy: 0.9573 - val_loss: 0.1963 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 110/500
26/26 [==============================] - 0s 5ms/step - loss: 0.1844 - accuracy: 0.9573 - val_loss: 0.1947 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 111/500
26/26 [==============================] - 0s 5ms/step - loss: 0.1826 - accuracy: 0.9567 - val_loss: 0.1929 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 112/500
26/26 [==============================] - 0s 5ms/step - loss: 0.1806 - accuracy: 0.9579 - val_loss: 0.1917 - val_accuracy: 0.9500 - lr: 0.0010
Epoch 113/500
26/26 [==============================] - 0s 5ms/st

<IPython.core.display.Javascript object>